In [2]:
import requests
import xml.etree.ElementTree as ET

In [3]:
DELTA = 0.01

In [4]:
def create_bounding_box(latitude, longitude, delta):
      # Calculate the latitude and longitude for the upper-left corner of the bounding box
    upper_left_lat = round(latitude - delta, 7)
    upper_left_lon = round(longitude - delta, 7)

    # Calculate the latitude and longitude for the lower-right corner of the bounding box
    lower_right_lat = round(latitude + delta, 7)
    lower_right_lon = round(longitude + delta, 7)
    return (upper_left_lat, upper_left_lon, lower_right_lat, lower_right_lon)

In [5]:
def fetch_data(api_url):
    response = requests.get(api_url)
    return response.json()


In [6]:
def fetch_data_xml(api_url):
    response = requests.get(api_url)
    root = ET.fromstring(response.content)
    return root

In [13]:
def getLatitudLongitud(ciudad):
    URL = 'https://nominatim.openstreetmap.org/search?q='+ciudad+'&format=json'
    data = fetch_data(URL)
    lat = data[0]['lat']
    #cast to float
    lat = float(lat)
    lon = data[0]['lon']
    #cast to float
    lon = float(lon)
    return lat, lon

def getDailyWeather(lat, lon):
    #https://api.open-meteo.com/v1/forecast?latitude=-12.04&longitude=-77.03&forecast_days=2&daily=temperature_2m_max&timezone=PST
    URL = 'https://api.open-meteo.com/v1/forecast?latitude='+str(lat)+'&longitude='+str(lon)+'&forecast_days=2&daily=temperature_2m_max&timezone=PST'
    data = fetch_data(URL) #temperature_2m_max
    # pair up (time,temperature_2m_max)
    result = []
    for i in range(len(data['daily']['temperature_2m_max'])):
        obj = {}
        key = data['daily']['time'][i]
        obj[key] = data['daily']['temperature_2m_max'][i]
        result.append(obj)
        
    return result

def getHourlyWeather(lat, lon):
    #https://api.open-meteo.com/v1/forecast?latitude=-12.04&longitude=-77.03&forecast_days=2&hourly=temperature_2m&timezone=PST
    URL = 'https://api.open-meteo.com/v1/forecast?latitude='+str(lat)+'&longitude='+str(lon)+'&forecast_days=2&hourly=temperature_2m&timezone=PST'
    data = fetch_data(URL)#temperature_2m
    # pair up (time,temperature_2m)
    result = []
    for i in range(len(data['hourly']['temperature_2m'])):
        obj = {}
        key = data['hourly']['time'][i]
        obj[key] = data['hourly']['temperature_2m'][i]
        result.append(obj)
    return result

In [14]:
lat,long = getLatitudLongitud('jesus maria lima')
print(lat,long)
daily = getDailyWeather(lat, long)
hourly = getHourlyWeather(lat, long)
print(daily) #temperature_2m_max
print(hourly) #temperature_2m



-12.0781861 -77.04640320962204
[{'2023-10-25': 19.7}, {'2023-10-26': 19.8}]
[{'2023-10-25T00:00': 16.9}, {'2023-10-25T01:00': 16.9}, {'2023-10-25T02:00': 17.0}, {'2023-10-25T03:00': 17.1}, {'2023-10-25T04:00': 17.2}, {'2023-10-25T05:00': 17.0}, {'2023-10-25T06:00': 17.3}, {'2023-10-25T07:00': 17.8}, {'2023-10-25T08:00': 18.6}, {'2023-10-25T09:00': 19.2}, {'2023-10-25T10:00': 19.7}, {'2023-10-25T11:00': 19.2}, {'2023-10-25T12:00': 18.9}, {'2023-10-25T13:00': 18.4}, {'2023-10-25T14:00': 18.0}, {'2023-10-25T15:00': 17.6}, {'2023-10-25T16:00': 17.1}, {'2023-10-25T17:00': 16.8}, {'2023-10-25T18:00': 16.7}, {'2023-10-25T19:00': 16.7}, {'2023-10-25T20:00': 16.7}, {'2023-10-25T21:00': 16.6}, {'2023-10-25T22:00': 16.6}, {'2023-10-25T23:00': 16.6}, {'2023-10-26T00:00': 16.7}, {'2023-10-26T01:00': 16.7}, {'2023-10-26T02:00': 16.7}, {'2023-10-26T03:00': 16.7}, {'2023-10-26T04:00': 16.7}, {'2023-10-26T05:00': 16.9}, {'2023-10-26T06:00': 17.5}, {'2023-10-26T07:00': 18.1}, {'2023-10-26T08:00': 18.8},

In [76]:
box = create_bounding_box(lat,long,DELTA)
print(box)

(-12.0881861, -77.0564032, -12.0681861, -77.0364032)


In [46]:
#bien
#https://api.openstreetmap.org/api/0.6/map?bbox=-77.0564032,-12.0881861,-77.0564032,-12.0781861
#mal
#https://api.openstreetmap.org/api/0.6/map?bbox=-77.0364032,-12.0881861,-77.0364032,-12.0881861

In [80]:
def getResutaranteBYBox(box):
    URL = 'https://api.openstreetmap.org/api/0.6/map?bbox='+str(box[1])+','+str(box[0])+','+str(box[3])+','+str(box[2])
    print(URL)
    data = fetch_data_xml(URL)
    return data

In [84]:
res = getResutaranteBYBox(box)
print(res)

https://api.openstreetmap.org/api/0.6/map?bbox=-77.0564032,-12.0881861,-77.0364032,-12.0681861
<Element 'osm' at 0x7ff04e863630>


In [121]:
restaurants= []
for element in res:
    # print(element.attrib)
    for tag in element:
        #si tiene tag k 
        if 'k' in tag.attrib:
            #si el tag k es amenity y el tag v es restaurant
            if tag.attrib['k'] == 'amenity' and tag.attrib['v'] == 'restaurant':
                restaurants.append(element)
            #print all tags 

# cada tag convierte en un objeto
rdata =[]
# Find restaurant elements in the XML
for element in res:
    restaurant_info = {}
    for tag in element:
        if 'k' not in tag.attrib or 'v' not in tag.attrib:
            continue
        k = tag.attrib['k']
        v = tag.attrib['v']
        restaurant_info[k] = v

    if 'amenity' in restaurant_info and restaurant_info['amenity'] == 'restaurant':
        rdata.append(restaurant_info)
   
        #     obj[k]=v
    # rdata.append(obj)

print(rdata)

[{'amenity': 'restaurant', 'cuisine': 'indian', 'name': 'Tajmahal Restaurant'}, {'amenity': 'restaurant', 'cuisine': 'sandwich,ice_cream,salchipapa;fried_chicke', 'name': 'Tia Veneno', 'source': 'Reconocimiento cartográfico por KG 2016.'}, {'addr:street': 'Jirón Pablo Bermudez', 'amenity': 'restaurant', 'name': 'Pollería Kendal'}, {'addr:city': 'Jesús María', 'addr:street': 'Avenida Francisco Javier Mariátegui', 'amenity': 'restaurant', 'capacity': '20', 'cuisine': 'regional', 'internet_access': 'wlan', 'name': 'Wira Wira'}, {'addr:housenumber': '311', 'addr:street': 'Avenida Simon Bolivar', 'amenity': 'restaurant', 'name': 'Ohashi Maki', 'opening_hours': 'Mo-Su 12:00-23:00', 'phone': '+51 1 5923284'}, {'addr:street': 'Avenida Salaverry', 'amenity': 'restaurant', 'name': 'Pio Pio'}, {'addr:street': 'Avenida 6 de Agosto', 'amenity': 'restaurant', 'name': "Norky's"}, {'addr:street': 'Avenida Cuba', 'amenity': 'restaurant', 'name': 'El Fogón Criollos', 'source': 'Reconocimiento cartográfi

In [122]:
def dict_to_xml(element, dictionary):
    for key, value in dictionary.items():
        sub_element = ET.SubElement(element, key)
        if isinstance(value, dict):
            dict_to_xml(sub_element, value)
        else:
            sub_element.text = str(value)

In [124]:
# Create a root element for the XML document
root = ET.Element("root")

# Iterate over the restaurant information and create XML elements
for restaurant_info in rdata:
    element = ET.Element("element")
    
    # Create XML sub-elements for each key-value pair in the restaurant information
    for key, value in restaurant_info.items():
        tag = ET.Element("tag", attrib={"k": key, "v": value})
        element.append(tag)
    
    root.append(element)

# Create an ElementTree and write it to an XML file
tree = ET.ElementTree(root)
print(tree)
tree.write("restaurants.xml")